# Import the libraries

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PART 1

## Import the sales and stores files for adidas

- Adidas_Sales.csv

- Adidas_Stores.csv

In [2]:
# Load in the entire data set from the Sales file
df_sales = pd.read_csv(filepath_or_buffer='../Data/Adidas_Sales.csv')

# Load in only a selected subset of columns from the Stores file
cols = ['Asset ID', 'Location', 'Province', 'City', 'Asset Type','Employees']
df_stores = pd.read_csv(filepath_or_buffer='../Data/Adidas_Stores.csv', index_col='Asset ID', usecols=cols)


## Merge both datasets into a single dataset

In [3]:
# Merge the Stores DataFrame into the Reporting DataFrame
df_merged = pd.merge(df_stores, df_sales, on='Asset ID', how='outer')


# Part 2 - Build a multi index

Use the columns 
- Asset ID
- Location
- Province
- Cit
- Asset Type
- Brand
- Financials

In [4]:
tuples = list(zip(df_merged['Asset ID'],df_merged['Location'], df_merged['Province'],
                  df_merged['City'], df_merged['Asset Type'], df_merged['Brand'], df_merged['Financials']))
hier_index = pd.MultiIndex.from_tuples(tuples)    

# Reset the Index
df_merged.reset_index()
df_merged.index = hier_index

# Drop unwanted columns
df_merged.pop('Asset ID')
df_merged.pop('Location')
df_merged.pop('Province')
df_merged.pop('City')
df_merged.pop('Asset Type')
df_merged.pop('Brand')
df_merged.pop('Financials')

df_merged.index.names = ['Asset ID','Location', 'Province', 'City', 'Asset Type', 'Brand', 'Financials']


# Part 3 - Pivot Tables

Using the same Multi Index created for part 2, produce pivots for the following

1)  Produce pivots for the following
    - by Asset ID
    - by Location
    - by Province
    - by City
    - by Asset Type
    - by Brand
    - by Financials

2) Create a pivot of the following
    - Location, Province, Asset Type
    - For each quarter between Mar-16 and Dec-17 (inclusive)

In [5]:
pd.options.display.float_format = '{:,.0f}'.format
cols = df_merged.columns[1:]

# by Asset ID
df_merged.pivot_table(index='Asset ID')[cols]

# by Location
df_merged.pivot_table(index='Location')[cols]

# by Province
df_merged.pivot_table(index='Province')[cols]

# by City
df_merged.pivot_table(index='City')[cols]

# by Asset Type
df_merged.pivot_table(index='Asset Type')[cols]

# by Brand
df_merged.pivot_table(index='Brand')[cols]

# by Financials
df_merged.pivot_table(index='Financials')[cols]


# by Location, Province and Asset Type
# For the quarters Mar-16 to Dec-17

idx = ['Location', 'Province', 'Asset Type']
cols = df_merged.columns[9:17]
df_merged.pivot_table(index=idx)[cols]


Mar-16  \
Location  Province                         Asset Type                      
Argentina Buenos Aires                     Distribution Centre   123,866   
                                           Store                   8,587   
          Ciudad Autonoma de Buenoas Aires Distribution Centre   188,544   
          Ciudad Autonoma de Buenos Aires  Distribution Centre   271,598   
Austria   (blank)                          Distribution Centre    91,494   
...                                                                  ...   
Vietnam   Thu Do Ha Noi                    Store                  49,400   
          Tien Giang                       Distribution Centre   414,242   
                                           Factory             1,400,597   
          Vinh Long                        Factory             6,464,714   
          Vinh Phuc                        Distribution Centre    17,995   

                                                                  Jun-16  \
Location  Province                         Asset Type                      
Argentina Buenos Aires                     Distribution Centre    89,080   
                                           Store                   8,894   
          Ciudad Autonoma de Buenoas Aires Distribution Centre   252,260   
          Ciudad Autonoma de Buenos Aires  Distribution Centre   247,522   
Austria   (blank)                          Distribution Centre    93,201   
...                                                                  ...   
Vietnam   Thu Do Ha Noi                    Store                  60,681   
          Tien Giang                       Distribution Centre   366,386   
                                           Factory             1,248,602   
          Vinh Long                        Factory             4,490,358   
          Vinh Phuc                        Distribution Centre    57,549   

                                                                  Sep-16  \
Location  Province                         Asset Type                      
Argentina Buenos Aires                     Distribution Centre    45,128   
                                           Store                  10,194   
          Ciudad Autonoma de Buenoas Aires Distribution Centre   209,950   
          Ciudad Autonoma de Buenos Aires  Distribution Centre   267,526   
Austria   (blank)                          Distribution Centre   116,496   
...                                                                  ...   
Vietnam   Thu Do Ha Noi                    Store                  49,399   
          Tien Giang                       Distribution Centre   426,754   
                                           Factory             1,341,781   
          Vinh Long                        Factory             6,663,932   
          Vinh Phuc                        Distribution Centre    69,000   

                                                                  Dec-16  \
Location  Province                         Asset Type                      
Argentina Buenos Aires                     Distribution Centre    56,707   
                                           Store                   9,940   
          Ciudad Autonoma de Buenoas Aires Distribution Centre   224,737   
          Ciudad Autonoma de Buenos Aires  Distribution Centre   243,542   
Austria   (blank)                          Distribution Centre   121,234   
...                                                                  ...   
Vietnam   Thu Do Ha Noi                    Store                  54,366   
          Tien Giang                       Distribution Centre   350,254   
                                           Factory             1,325,196   
          Vinh Long                        Factory             5,738,534   
          Vinh Phuc                        Distribution Centre   151,675   

                                                                  Mar-17  \
Location  Province                

# Part 4 - Group By

Using the same Multi Index created for part 2, use the pandas.groupby method to produce groups of the following

1) The sum of all Financials for each Location for the quarters Mar-16 through to Dec-17 inclusive

2) The total number of employees for the entire enterprise grouped by Lcoation, Brand and City 



In [6]:
df_merged.groupby(level=['Location', 'Financials']).sum().loc[:,'Mar-16':'Dec-17']


Mar-16     Jun-16     Sep-16  \
Location  Financials                                                   
Argentina Cost of Sales                1128243    1189598    1131810   
          Gross Profit                  704303     693976     619473   
          Income Before Tax             658862     643327     558838   
          Net Income                    575450     556720     473301   
          Net Sales                    1832546    1883574    1751283   
...                                        ...        ...        ...   
Vietnam   Net Sales                  175353611  172186946  179373813   
          Operating Income            51786843   48025823   58156818   
          Provisions for Income Tax   11387586   11360594   11386489   
          Total Operating Expenses    15005855   15049361   14795326   
          Total Other Income          10238586   10205367   10082211   

                                        Dec-16     Mar-17     Jun-17  \
Location  Financials                                                   
Argentina Cost of Sales                1147278    1208201    1231997   
          Gross Profit                  602103     595159     645511   
          Income Before Tax             565981     546259     597001   
          Net Income                    481876     457625     511582   
          Net Sales                    1749381    1803360    1877508   
...                                        ...        ...        ...   
Vietnam   Net Sales                  172713541  177220299  173191210   
          Operating Income            49310160   55226546   49244113   
          Provisions for Income Tax   11042363   11168383   11616700   
          Total Operating Expenses    15076024   14662891   14945673   
          Total Other Income          10394660   10232060   10167758   

                                        Sep-17     Dec-17  
Location  Financials                                       
Argentina Cost of Sales                1129421    1279568  
          Gross Profit                  864758     725468  
          Income Before Tax             818845     659036  
          Net Income                    736460     576609  
          Net Sales                    1994179    2005036  
...                                        ...        ...  
Vietnam   Net Sales                  176482566  173268458  
          Operating Income            50996166   52124985  
          Provisions for Income Tax   11402937   11296425  
          Total Operating Expenses    15147393   14947504  
          Total Other Income          10637484   10307623  

[531 rows x 8 columns]

In [7]:
grps = ['Location', 'Brand','City']


df_merged['Employees'].groupby(by=grps).sum()


Location   Brand   City                    
Argentina  Adidas  Buenos Aires                 108
                   Coronel Suárez               729
                   Haedo                        171
                   Las Flores                    54
                   Partido de Vicente Lopez      36
                                               ... 
Vietnam    Reebok  Tan Phuoc District           738
                   Thai Nguyen                  891
                   Tien Giang                 1,422
                   Trieu Son                  1,980
                   Vinh City                  1,620
Name: Employees, Length: 672, dtype: float64

# Part 5 - Cross Sections

Use the pandas.xs method to produce the following cross sections

1) Total quarterly Financials for the USA for the year 2018

2) Total quarterly Operating Income for each country for the year 2017

In [8]:
# the total Financials for the United States for the year 2018
df_merged.xs(key='United States', level='Location', drop_level=False).loc[:,'Mar-18' : 'Dec-18'].groupby(level=['Location', 'Financials']).sum()


Mar-18   Jun-18   Sep-18   Dec-18
Location      Financials                                                   
United States Cost of Sales              2089755  2131428  2091859  2087261
              Gross Profit               1508690  1511672  1420784  1629611
              Income Before Tax          1426595  1434623  1324051  1546299
              Net Income                 1237347  1248438  1138032  1352001
              Net Sales                  3598445  3643100  3512643  3716872
              Operating Income           1200151  1212183  1101003  1327002
              Provisions for Income Tax   189248   186185   186019   194298
              Total Operating Expenses    308539   299489   319781   302609
              Total Other Income          226444   222440   223048   219297

In [9]:
# The operating income for all countries
df_merged.xs(key='Operating Income', level='Financials', drop_level=False).loc[:,'Mar-17' : 'Dec-17'].groupby(level=['Location', 'Financials']).sum()


,,Mar-17,Jun-17,Sep-17,Dec-17
Location,Financials,,,,
Argentina,Operating Income,427916,486571,697292,551379
Austria,Operating Income,-4309,88643,47284,17326
Bangladesh,Operating Income,7839404,3782009,8211377,6210371
Belgium,Operating Income,66688,38589,39918,58904
Bosnia And Herzegovina,Operating Income,204035,151423,167023,217911
Brazil,Operating Income,1594461,2255155,1968203,1886706
Cambodia,Operating Income,4587634,3410445,3271195,3629171
Canada,Operating Income,-107190,-129992,-132063,-54561
Chile,Operating Income,18616,18220,15101,12713
